<a href="https://colab.research.google.com/github/Jii111/ESAA-2/blob/main/4%EC%A3%BC%EC%B0%A8%20%EC%84%B8%EC%85%98%20%EC%95%99%EC%83%81%EB%B8%94.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1-a. MNIST 데이터를 훈련, 검증, 테스트 데이터로 나눈다.

In [1]:
from sklearn.datasets import fetch_openml
mnist = fetch_openml('mnist_784',version=1)

In [2]:
from sklearn.model_selection import train_test_split

#val
X_train_val, X_test, y_train_val, y_test = train_test_split(mnist.data, mnist.target, test_size=10000)
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=10000)

1-b. 랜덤 포레스트 분류기, 엑스트라 트리 분류기, SVM 분류기, MLP 분류기를 훈련시킨다. (n_estimators=100, random_state=42)

In [3]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC

rnd_clf=RandomForestClassifier(n_estimators=100,random_state=42)
svm_clf=SVC(random_state=42,probability=True)



In [4]:
rnd_clf.fit(X_train,y_train)
svm_clf.fit(X_train,y_train)

SVC(probability=True, random_state=42)

In [5]:
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.neural_network import MLPClassifier

ext_clf=ExtraTreesClassifier(n_estimators=100,random_state=42)
mlp_clf=MLPClassifier(random_state=42)


In [6]:
ext_clf.fit(X_train,y_train)

ExtraTreesClassifier(random_state=42)

In [7]:
mlp_clf.fit(X_train,y_train)

MLPClassifier(random_state=42)

1-c. 이들을 직접 투표 분류기를 사용하는 앙상블로 연결한다.

In [8]:
from sklearn.ensemble import VotingClassifier

voting_clf=VotingClassifier(
    estimators=[('rf',rnd_clf),('svc',svm_clf),('exf',ext_clf),('mlf',mlp_clf)],
    voting='hard')



In [9]:
voting_clf.fit(X_train, y_train)

VotingClassifier(estimators=[('rf', RandomForestClassifier(random_state=42)),
                             ('svc', SVC(probability=True, random_state=42)),
                             ('exf', ExtraTreesClassifier(random_state=42)),
                             ('mlf', MLPClassifier(random_state=42))])

In [10]:
from sklearn.metrics import accuracy_score

for clf in (rnd_clf,svm_clf,ext_clf,mlp_clf,voting_clf):
  y_pred=clf.predict(X_test)
  print(clf.__class__.__name__,accuracy_score(y_test,y_pred))


RandomForestClassifier 0.9682
SVC 0.9772
ExtraTreesClassifier 0.9712
MLPClassifier 0.9628
VotingClassifier 0.9764


1-d. 다른 분류기보다 성능이 많이 떨어졌던 선형 SVM 제거 후 VotingClassifier를 다시 평가

In [11]:
voting_clf1=VotingClassifier(
    estimators=[('rf',rnd_clf),('exf',ext_clf),('mlf',mlp_clf)],
    voting='hard')

In [12]:
voting_clf1.fit(X_train, y_train)

VotingClassifier(estimators=[('rf', RandomForestClassifier(random_state=42)),
                             ('exf', ExtraTreesClassifier(random_state=42)),
                             ('mlf', MLPClassifier(random_state=42))])

In [14]:
for clf in (rnd_clf,svm_clf,ext_clf,mlp_clf,voting_clf1):
  y_pred=clf.predict(X_test)
  print(clf.__class__.__name__,accuracy_score(y_test,y_pred))

RandomForestClassifier 0.9682
SVC 0.9772
ExtraTreesClassifier 0.9712
MLPClassifier 0.9628
VotingClassifier 0.9734


1-d. 간접 투표 분류기를 사용하여 앙상블로 연결.

In [15]:
voting_clf2=VotingClassifier(
    estimators=[('rf',rnd_clf),('exf',ext_clf),('mlf',mlp_clf)],
    voting='soft')

In [21]:
voting_clf2.fit(X_train, y_train)

VotingClassifier(estimators=[('rf', RandomForestClassifier(random_state=42)),
                             ('exf', ExtraTreesClassifier(random_state=42)),
                             ('mlf', MLPClassifier(random_state=42))],
                 voting='soft')

In [ ]:
from sklearn.metrics import accuracy_score
for clf in (rnd_clf,ext_clf,mlp_clf,voting_clf2):
  y_pred=clf.predict(X_test)
  print(clf.__class__.__name__,accuracy_score(y_test,y_pred))